In [1]:
import sys
sys.path.append('./python')

import numpy as np
import pytest
from needle import backend_ndarray as nd
import needle.nn as nn
import needle.optim as optim
import needle as ndl
from needle import data as ndldata
import time

np.random.seed(4)

# Matmul test - CPU VS GPU

In [2]:
matmul_dims = (5000, 5000, 5000)

m = matmul_dims[0]
n = matmul_dims[1]
p = matmul_dims[2]

_A = np.random.randn(m, n)
_B = np.random.randn(n, p)

st = time.time()
_S = _A @ _B
ed = time.time()
total = ed - st

print("time of cpu:", total)

A = nd.array(_A, device = nd.cuda())
B = nd.array(_B, device = nd.cuda())

st = time.time()
S = A @ B
ed = time.time()
total = ed - st

print("time of gpu:", total)

time of cpu: 5.98693323135376
time of gpu: 0.004266977310180664


# Inference MNIST On CPU

In [3]:
import sys
sys.path.append('./python')

import numpy as np
import pytest
import needle as ndl
from needle import backend_ndarray as nd
import needle.nn as nn
import needle.optim as optim
from needle import data as ndldata
import needle.init as init
import needle.ops as ops
import time
from tqdm import tqdm

np.random.seed(4)
# model
def MLPNet(dim, hidden_dim=100, num_classes=10):
    ### BEGIN YOUR SOLUTION
    mlpNet = nn.Sequential(
        nn.Linear(in_features=dim, out_features=hidden_dim), 
        nn.ReLU(), 
        nn.Linear(in_features=hidden_dim, out_features=num_classes))
    return mlpNet

BATCH_SIZE = 100
HIDDEN_DIM = 100

mnist_train_dataset = ndldata.MNISTDataset("data/train-images-idx3-ubyte.gz",
                                            "data/train-labels-idx1-ubyte.gz")

mnist_train_dataloader = ndldata.DataLoader(dataset=mnist_train_dataset,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True)

mnist_test_dataset = ndldata.MNISTDataset("data/t10k-images-idx3-ubyte.gz",
                                               "data/t10k-labels-idx1-ubyte.gz")

mnist_test_dataloader = ndldata.DataLoader(dataset=mnist_test_dataset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=False)

model = MLPNet(784, HIDDEN_DIM)

loss_fn = nn.SoftmaxLoss()
loss_list = []
acc_list = []

model.train()

opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

In [4]:
n_sample, correct = 0, 0
for i, batch in tqdm(enumerate(mnist_train_dataloader)):
    x, y = batch[0], batch[1]
    x = x.reshape((x.shape[0], -1))
    y_hat = model(x)
    
    loss = loss_fn(y_hat, y)
    loss_list.append(loss.cached_data.numpy())
    
    logit = nd.NDArray(np.argmax(y_hat.cached_data.numpy(), axis=1))
    correct += np.sum((logit == y.cached_data).numpy().astype('int'))
    
    n_sample += x.shape[0]
    acc_list.append(correct / x.shape[0])
    
    opt.reset_grad()
    loss.backward()
    opt.step()
    
acc, loss = correct/n_sample, np.mean(loss_list)


600it [00:29, 20.27it/s]


In [7]:
acc

0.9009666666666667